In [1]:
#-*- coding:utf-8 -*-  
import logging  
import logging.config  
import configparser  
import numpy as np  
import random  
import codecs  
import os  
import pandas as pd
import jieba.analyse
import re
import random
import time
from collections import OrderedDict 

In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("LDA module")
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
formatter1 = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.ERROR)
logger.addHandler(ch)
fh = logging.FileHandler("log/LDA_log.txt",encoding = 'utf-8')
fh.setLevel(logging.INFO)
logger.addHandler(fh)
ch.setFormatter(formatter1)
fh.setFormatter(formatter)


In [3]:
pd.set_option('max_colwidth',100000)
# jieba.analyse.set_stop_words("data/stopwords.txt")

In [4]:
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords

In [5]:
def seg_sentence(sentence):  
    sentence_seged = jieba.cut(sentence.strip())  
    stopwords = stopwordslist('data/stopwords.txt')  # 这里加载停用词的路径  
    outstr = ''  
    for word in sentence_seged:  
        if word not in stopwords:  
            if word != '\t':  
                outstr += word  
                outstr += " "  
    return outstr  

In [6]:
path = os.getcwd()
conf = configparser.ConfigParser()  
conf.read("tool/setting.conf")  

['tool/setting.conf']

In [7]:
#文件路径  
trainfile = os.path.join(path,os.path.normpath(conf.get("filepath", "trainfile")))  
wordidmapfile = os.path.join(path,os.path.normpath(conf.get("filepath","wordidmapfile")))  
thetafile = os.path.join(path,os.path.normpath(conf.get("filepath","thetafile")))  
phifile = os.path.join(path,os.path.normpath(conf.get("filepath","phifile")))  
paramfile = os.path.join(path,os.path.normpath(conf.get("filepath","paramfile")))  
topNfile = os.path.join(path,os.path.normpath(conf.get("filepath","topNfile")))  
tassginfile = os.path.join(path,os.path.normpath(conf.get("filepath","tassginfile")))

In [8]:
#模型初始参数  
K = int(conf.get("model_args","K"))  
alpha = float(conf.get("model_args","alpha"))  
beta = float(conf.get("model_args","beta"))  
iter_times = int(conf.get("model_args","iter_times"))  
top_words_num = int(conf.get("model_args","top_words_num"))  

In [15]:
class Document(object):  
    def __init__(self):  
        self.words = []  
        self.length = 0  

In [9]:
alpha

0.1

In [10]:
#把整个文档及真的单词构成vocabulary（不允许重复）  
class DataPreProcessing(object):  
    def __init__(self):  
        self.docs_count = 0  
        self.words_count = 0  
        #保存每个文档d的信息(单词序列，以及length)  
        self.docs = []  
        #建立vocabulary表，照片文档的单词  
        self.word2id = OrderedDict()  
    def cachewordidmap(self):  
        with codecs.open(wordidmapfile, 'w','utf-8') as f:  
            for word,id in self.word2id.items():  
                f.write(word +"\t"+str(id)+"\n")  

In [11]:
class LDAModel(object):  
    def __init__(self,dpre):  
        self.dpre = dpre #获取预处理参数  
        #  
        #模型参数  
        #聚类个数K，迭代次数iter_times,每个类特征词个数top_words_num,超参数α（alpha） β(beta)  
        #  
        self.K = K  
        self.beta = beta  
        self.alpha = alpha  
        self.iter_times = iter_times  
        self.top_words_num = top_words_num   
        #  
        #文件变量  
        #分好词的文件trainfile  
        #词对应id文件wordidmapfile  
        #文章-主题分布文件thetafile  
        #词-主题分布文件phifile  
        #每个主题topN词文件topNfile  
        #最后分派结果文件tassginfile  
        #模型训练选择的参数文件paramfile  
        #  
        self.wordidmapfile = wordidmapfile  
        self.trainfile = trainfile  
        self.thetafile = thetafile  
        self.phifile = phifile  
        self.topNfile = topNfile  
        self.tassginfile = tassginfile  
        self.paramfile = paramfile  
        # p,概率向量 double类型，存储采样的临时变量  
        # nw,词word在主题topic上的分布  
        # nwsum,每各topic的词的总数  
        # nd,每个doc中各个topic的词的总数  
        # ndsum,每各doc中词的总数  
        self.p = np.zeros(self.K)  
        # nw,词word在主题topic上的分布  
        self.nw = np.zeros((self.dpre.words_count,self.K),dtype="int")  
        # nwsum,每各topic的词的总数  
        self.nwsum = np.zeros(self.K,dtype="int")  
        # nd,每个doc中各个topic的词的总数  
        self.nd = np.zeros((self.dpre.docs_count,self.K),dtype="int")  
        # ndsum,每各doc中词的总数  
        self.ndsum = np.zeros(dpre.docs_count,dtype="int")  
        self.Z = np.array([ [0 for y in range(dpre.docs[x].length)] for x in range(dpre.docs_count)])        # M*doc.size()，文档中词的主题分布  
  
        #随机先分配类型，为每个文档中的各个单词分配主题  
        for x in range(len(self.Z)):  
            self.ndsum[x] = self.dpre.docs[x].length  
            for y in range(self.dpre.docs[x].length):  
                topic = random.randint(0,self.K-1)#随机取一个主题  
                self.Z[x][y] = topic#文档中词的主题分布  
                self.nw[self.dpre.docs[x].words[y]][topic] += 1  
                self.nd[x][topic] += 1  
                self.nwsum[topic] += 1  
  
        self.theta = np.array([ [0.0 for y in range(self.K)] for x in range(self.dpre.docs_count) ])  
        self.phi = np.array([ [ 0.0 for y in range(self.dpre.words_count) ] for x in range(self.K)])   
    def sampling(self,i,j):  
        #换主题  
        topic = self.Z[i][j]  
        #只是单词的编号，都是从0开始word就是等于j  
        word = self.dpre.docs[i].words[j]  
        #if word==j:  
        #    print 'true'  
        self.nw[word][topic] -= 1  
        self.nd[i][topic] -= 1  
        self.nwsum[topic] -= 1  
        self.ndsum[i] -= 1  
  
        Vbeta = self.dpre.words_count * self.beta  
        Kalpha = self.K * self.alpha  
        self.p = (self.nw[word] + self.beta)/(self.nwsum + Vbeta) * (self.nd[i] + self.alpha) / (self.ndsum[i] + Kalpha)  
  
        #随机更新主题的吗  
        # for k in range(1,self.K):  
        #     self.p[k] += self.p[k-1]  
        # u = random.uniform(0,self.p[self.K-1])  
        # for topic in range(self.K):  
        #     if self.p[topic]>u:  
        #         break  
  
        #按这个更新主题更好理解，这个效果还不错  
        p = np.squeeze(np.asarray(self.p/np.sum(self.p)))  
        topic = np.argmax(np.random.multinomial(1, p))  
  
        self.nw[word][topic] +=1  
        self.nwsum[topic] +=1  
        self.nd[i][topic] +=1  
        self.ndsum[i] +=1  
        return topic  
    def est(self,text_id):  
        # Consolelogger.info(u"迭代次数为%s 次" % self.iter_times)  
        for x in range(self.iter_times):  
            for i in range(self.dpre.docs_count):  
                for j in range(self.dpre.docs[i].length):  
                    topic = self.sampling(i,j)  
                    self.Z[i][j] = topic            
            logger.debug(u"计算文章-主题分布")  
            self._theta()  
            logger.debug(u"计算词-主题分布")  
            self._phi()  
        logger.info(u"迭代完成。")
        logger.debug(u"保存模型")  
        self.save(text_id)  
    def _theta(self):  
        for i in range(self.dpre.docs_count):#遍历文档的个数词  
            self.theta[i] = (self.nd[i]+self.alpha)/(self.ndsum[i]+self.K * self.alpha)  
    def _phi(self):  
        for i in range(self.K):  
            self.phi[i] = (self.nw.T[i] + self.beta)/(self.nwsum[i]+self.dpre.words_count * self.beta)  
    def save(self,text_id):  
        # 保存theta文章-主题分布  
        logger.info(u"文章-主题分布已保存到%s" % self.thetafile)  
        text_count = 0
        with codecs.open(self.thetafile,'w','utf-8') as f:  
            for x in range(self.dpre.docs_count):  
                f.write(str(text_id[text_count])+'\t\t')
                text_count += 1
                for y in range(self.K):  
                    f.write(str(self.theta[x][y]) + '\t')  
                f.write('\n')  
        # 保存phi词-主题分布  
        logger.info(u"词-主题分布已保存到%s" % self.phifile)  
        with codecs.open(self.phifile,'w') as f:  
            for x in range(self.K):  
                for y in range(self.dpre.words_count):  
                    f.write(str(self.phi[x][y]) + '\t')  
                f.write('\n')  
        # 保存参数设置  
        logger.info(u"参数设置已保存到%s" % self.paramfile)  
        with codecs.open(self.paramfile,'w','utf-8') as f:  
            f.write('K=' + str(self.K) + '\n')  
            f.write('alpha=' + str(self.alpha) + '\n')  
            f.write('beta=' + str(self.beta) + '\n')  
            f.write(u'迭代次数  iter_times=' + str(self.iter_times) + '\n')  
            f.write(u'每个类的高频词显示个数  top_words_num=' + str(self.top_words_num) + '\n')  
        # 保存每个主题topic的词  
        logger.info(u"主题topN词已保存到%s" % self.topNfile)  
  
        with codecs.open(self.topNfile,'w','utf-8') as f:  
            self.top_words_num = min(self.top_words_num,self.dpre.words_count)  
            for x in range(self.K):  
                f.write(u'第' + str(x) + u'类：' + '\n')  
                twords = []  
                twords = [(n,self.phi[x][n]) for n in range(self.dpre.words_count)]  
                twords.sort(key = lambda i:i[1], reverse= True)  
                for y in range(self.top_words_num):  
                    word = OrderedDict({value:key for key, value in self.dpre.word2id.items()})[twords[y][0]]  
                    f.write('\t'*2+ word +'\t' + str(twords[y][1])+ '\n')  
        # 保存最后退出时，文章的词分派的主题的结果  
        logger.info(u"文章-词-主题分派结果已保存到%s" % self.tassginfile)  
        with codecs.open(self.tassginfile,'w') as f:  
            for x in range(self.dpre.docs_count):  
                for y in range(self.dpre.docs[x].length):  
                    f.write(str(self.dpre.docs[x].words[y])+':'+str(self.Z[x][y])+ '\t')  
                f.write('\n')  
        logger.info(u"模型训练完成。")  

In [12]:
def preprocessing():  
    logger.info(u'载入数据......')  
    with codecs.open(trainfile, 'r','utf-8') as f:  
        docs = f.readlines()  
#     print('docs is ',docs)
    logger.debug(u"载入完成,准备生成字典对象和统计文本数据...")  
    # 大的文档集  
    dpre = DataPreProcessing()  
    items_idx = 0  
    text_id=[]
    for line in docs:  
        text_list = line.strip().split("\t\t")
#         print("text_list is",text_list)
        if len(text_list) > 1:  
            text_id.append(text_list[0])        
            key_list = jieba.analyse.extract_tags(text_list[1], 30)
            tmp = []
            for i in key_list:
                if i not in stopwordslist('data/stopwords.txt'):
                    tmp.append(i)
#             tmp = line.strip().split()  
#             print("tmp is ",tmp)
            # 生成一个文档对象：包含单词序列（w1,w2,w3,,,,,wn）可以重复的  
            doc = Document()  
            for item in tmp:  
                if item in dpre.word2id:# 已有的话，只是当前文档追加  
                    doc.words.append(dpre.word2id[item])  
                else:  # 没有的话，要更新vocabulary中的单词词典及wordidmap  
                    dpre.word2id[item] = items_idx  
                    doc.words.append(items_idx)  
                    items_idx += 1  
            doc.length = len(tmp)  
            dpre.docs.append(doc)  
        else:  
            pass  
    dpre.docs_count = len(dpre.docs) # 文档数  
    dpre.words_count = len(dpre.word2id) # 词汇数  
    logger.info(u"共有%s个文档" % dpre.docs_count)  
    dpre.cachewordidmap()  
    logger.info(u"词与序号对应关系已保存到%s" % wordidmapfile)  
    return dpre,text_id

In [13]:
def run():  
    # 处理文档集，及计算文档数，以及vocabulary词的总个数，以及每个文档的单词序列  
    dpre,text_id = preprocessing()  
    lda = LDAModel(dpre)  
    lda.est(text_id)  

In [16]:
if __name__ == '__main__':  
#     f1 = open('in_out/topNfile.txt','w','utf-8')
#     f1.truncate()
#     f1 = open('in_out/paramflie.txt')
    run()  

INFO:LDA module:载入数据......
DEBUG:LDA module:载入完成,准备生成字典对象和统计文本数据...
INFO:LDA module:共有72793个文档
INFO:LDA module:词与序号对应关系已保存到E:\pythonNotebook\LDA_model\in_out\wordidmap.txt
DEBUG:LDA module:计算文章-主题分布
DEBUG:LDA module:计算词-主题分布
DEBUG:LDA module:计算文章-主题分布
DEBUG:LDA module:计算词-主题分布
DEBUG:LDA module:计算文章-主题分布
DEBUG:LDA module:计算词-主题分布
DEBUG:LDA module:计算文章-主题分布
DEBUG:LDA module:计算词-主题分布
DEBUG:LDA module:计算文章-主题分布
DEBUG:LDA module:计算词-主题分布
DEBUG:LDA module:计算文章-主题分布
DEBUG:LDA module:计算词-主题分布
DEBUG:LDA module:计算文章-主题分布
DEBUG:LDA module:计算词-主题分布
DEBUG:LDA module:计算文章-主题分布
DEBUG:LDA module:计算词-主题分布
DEBUG:LDA module:计算文章-主题分布
DEBUG:LDA module:计算词-主题分布
DEBUG:LDA module:计算文章-主题分布
DEBUG:LDA module:计算词-主题分布
DEBUG:LDA module:计算文章-主题分布
DEBUG:LDA module:计算词-主题分布
DEBUG:LDA module:计算文章-主题分布
DEBUG:LDA module:计算词-主题分布
DEBUG:LDA module:计算文章-主题分布
DEBUG:LDA module:计算词-主题分布
DEBUG:LDA module:计算文章-主题分布
DEBUG:LDA module:计算词-主题分布
DEBUG:LDA module:计算文章-主题分布
DEBUG:LDA module:计算词-主题分布
DEBUG:LDA module:计算文章-主题分布
DEBUG: